# Create Slice for Fabric Rack
This is the slice creation for the ```Trafic Analysis``` tutorial, This is the first step of the assignment, to create the slice on fabric using fablib

## 1. Set up the Experiment

In this section you will use the Fablib maager to create a new slice that is composed of 2 nodes for the Traffic Analysis Assignment.
### 1.1 Reserve Resources
In this section we will declare the Structure of the Slice we will be using.

In [ ]:
# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                     
fablib.show_config()
import json
import traceback

In [ ]:
# Define and Submit Slice
slice_name = "TrafficAnalysis"
site = "EDUKY"
print(site)

nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 1
ram = 2
disk = 10
network_name = 'lan'

try:
    #Create Slice
    slice = fablib.new_slice(name = slice_name)
    
    #Server
    server = slice.add_node(name="Server", site=site)
    server.set_image(image)
    server_port = server.add_component(model=nicmodel, name="server_port").get_interfaces()[0] 
    
    #Client
    client = slice.add_node(name="Client", site=site)
    client.set_image(image)
    client_port = client.add_component(model=nicmodel, name="client_port").get_interfaces()[0] 
    
    #changed name
    lan = slice.add_l2network(name=network_name, interfaces=[server_port, client_port])
    
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Slice Failed: {e}")

### 1.2 Set up the Experiment Network
This section of the slice creation, once the nodes are active, will asign the network spaces some predefined values that we will use for the Assgnment

In [ ]:
# Setup Network
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

subnet = IPv4Network("10.1.1.0/24")
available_ips = list(subnet)[1:]


network_name = 'lan'

try:
    for node in slice.get_nodes():
        node_iface = node.get_interface(network_name=network_name)  
        node_addr = available_ips.pop(0)
        node_iface.ip_addr_add(addr=node_addr, subnet=subnet)
        
        for interface in node.get_interfaces():
            stdout, stderr = node.execute(f'sudo ip link set dev { interface.get_device_name()} up')
        print(f"Network status at node {node.get_name()}")
        node.execute("ip a")

        stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')

except Exception as e:
    print(f"Fail: {e}")

### 1.3 Configure the software needed for the nodes in the experiment
This section of the slice creation, we will Add any aditional software, tools or scripts that we need for our experiments.

In [ ]:
# Install Software
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:    
    for node in slice.get_nodes():
        node.execute("sudo apt install net-tools")
        stdout, stderr = node.execute(f'sudo apt-get update')
        stdout, stderr = node.execute(f'sudo apt-get install iperf')
except Exception as e:
    print(f"Exception: {e}")

## Continue to The experiment notebook

Once You have completed this notebook you should be able to continue to the Traffic Analysis, you can ether open it on the explorer or click [Here](./Traffic_Analysis.ipynb) to open the next notebook.